# Pre-Processing

In [2]:
import pandas as pd

In [3]:
# import dataset
df = pd.read_csv(r'C:\Users\kettin\Documents\Lomba\GEMASTIKSciPaper\Dataset\ENSO_dataset.csv')
print(df.head())

   Tahun  Bulan  Grid1  Grid2  Grid3  Grid4  Grid5  Grid6  Grid7  Grid8  ...  \
0   1950      1    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
1   1950      2    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
2   1950      3    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
3   1950      4    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   
4   1950      5    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  ...   

   Grid16013  Grid16014  Grid16015  Grid16016  Grid16017  Grid16018  \
0       -1.8       -1.8       -1.8       -1.8       -1.8       -1.8   
1       -1.8       -1.8       -1.8       -1.8       -1.8       -1.8   
2       -1.8       -1.8       -1.8       -1.8       -1.8       -1.8   
3       -1.8       -1.8       -1.8       -1.8       -1.8       -1.8   
4       -1.8       -1.8       -1.8       -1.8       -1.8       -1.8   

   Grid16019  Grid16020  NINO3.4  ANOM  
0       -1.8       -1.8    24.55 -1.55  
1       -1

In [4]:
#hapus semua kolom yang isinya NULL semua
print(df.columns[df.isna().all()])
df = df.dropna(axis=1, how='all')

Index(['Grid1', 'Grid2', 'Grid3', 'Grid4', 'Grid5', 'Grid6', 'Grid7', 'Grid8',
       'Grid9', 'Grid10',
       ...
       'Grid15459', 'Grid15460', 'Grid15461', 'Grid15462', 'Grid15463',
       'Grid15464', 'Grid15465', 'Grid15466', 'Grid15467', 'Grid15468'],
      dtype='object', length=5032)


In [5]:
#cek jumlah NULL tersisa
total_nan = df.isna().sum().sum()
print(f"Jumlah total NaN di seluruh dataset: {total_nan}")

Jumlah total NaN di seluruh dataset: 0


In [6]:
#karena yang digunakan untuk menentukan el nino la nina adalah anomalinya
# Hapus kolom NINO3.4 lama
df = df.drop(columns=['NINO3.4'])

# Ganti nama ANOM jadi NINO3.4
df = df.rename(columns={'ANOM': 'NINO3.4'})

In [7]:
#cek apakah grid berisi suhu asli atau anomali
# Ambil semua kolom grid (selain Tahun, Bulan, NINO3.4)
grid_cols = df.drop(columns=['Tahun', 'Bulan', 'NINO3.4']).columns

import numpy as np

# Ambil semua nilai grid ke array 1D
all_values = df[grid_cols].values.flatten()

# Cari min, max, dan mean
min_val = np.min(all_values)
max_val = np.max(all_values)
mean_val = np.mean(all_values)

print("Min:", min_val)
print("Max:", max_val)
print("Mean:", mean_val)


Min: -1.7999999523162842
Max: 34.39228820800781
Mean: 13.74652894268218


In [8]:
#karena masih meruapakan suhu mentah, kita ubah grid SST menjadi anomali

# 1. Filter data baseline (1981–2010)
baseline_df = df[(df['Tahun'] >= 1981) & (df['Tahun'] <= 2010)]

# 2. Hitung klimatologi bulanan (rata-rata setiap bulan per grid) dari baseline
climatology = baseline_df.groupby('Bulan')[grid_cols].mean()

# 3. Hitung anomali untuk seluruh dataset (1950–2024)
def calc_anomaly(row):
    month = row['Bulan']
    return row[grid_cols] - climatology.loc[month]

df[grid_cols] = df.apply(calc_anomaly, axis=1)

print(df.head())


   Tahun  Bulan  Grid1163  Grid1164  Grid1165  Grid1166  Grid1167  Grid1168  \
0   1950      1  0.301079 -0.090344 -0.289011 -0.388230 -0.407921 -0.379495   
1   1950      2  0.024788 -0.230754 -0.334247 -0.412546 -0.427949 -0.382773   
2   1950      3  0.102170 -0.047959 -0.084494 -0.207032 -0.268234 -0.207919   
3   1950      4 -0.008598 -0.000866 -0.000061 -0.000299 -0.001452 -0.001958   
4   1950      5  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

   Grid1169  Grid1170  ...  Grid16012  Grid16013  Grid16014  Grid16015  \
0 -0.351447 -0.305173  ...        0.0        0.0        0.0        0.0   
1 -0.353728 -0.336454  ...        0.0        0.0        0.0        0.0   
2 -0.144358 -0.096469  ...        0.0        0.0        0.0        0.0   
3 -0.001467 -0.001367  ...        0.0        0.0        0.0        0.0   
4  0.000000  0.000000  ...        0.0        0.0        0.0        0.0   

   Grid16016  Grid16017  Grid16018  Grid16019  Grid16020  NINO3.4  
0        0.0

In [9]:
# Gabungkan semua nilai grid ke satu array
all_values = df[grid_cols].values.flatten()

# Hitung jumlah 0.0 dan total cell
zero_count = np.sum(all_values == 0.0)
total_count = all_values.size

# Hitung persentase
percent_zero = (zero_count / total_count) * 100

print(f"Jumlah cell 0.0: {zero_count}")
print(f"Total cell: {total_count}")
print(f"Persentase 0.0: {percent_zero:.2f}%")

Jumlah cell 0.0: 1198868
Total cell: 9889200
Persentase 0.0: 12.12%


In [10]:
from sklearn.preprocessing import MinMaxScaler

# Normalisasi grid SST ke 0–1 (langsung overwrite)
scaler = MinMaxScaler()
df[grid_cols] = scaler.fit_transform(df[grid_cols])

# Cek hasil
print(df.head())

   Tahun  Bulan  Grid1163  Grid1164  Grid1165  Grid1166  Grid1167  Grid1168  \
0   1950      1  0.681309  0.588332  0.442800  0.452465  0.508452  0.574813   
1   1950      2  0.498810  0.481532  0.409890  0.434850  0.496863  0.573238   
2   1950      3  0.549923  0.620571  0.591584  0.583730  0.589284  0.657282   
3   1950      4  0.476758  0.656391  0.653008  0.733495  0.743660  0.756278   
4   1950      5  0.482436  0.657050  0.653053  0.733711  0.744500  0.757219   

   Grid1169  Grid1170  ...  Grid16012  Grid16013  Grid16014  Grid16015  \
0  0.603147  0.611256  ...   0.010061        0.0        0.0        0.0   
1  0.602100  0.596741  ...   0.010061        0.0        0.0        0.0   
2  0.698252  0.708102  ...   0.010061        0.0        0.0        0.0   
3  0.763874  0.752233  ...   0.010061        0.0        0.0        0.0   
4  0.764548  0.752867  ...   0.010061        0.0        0.0        0.0   

   Grid16016  Grid16017  Grid16018  Grid16019  Grid16020  NINO3.4  
0        0.0

In [11]:
# Simpan ke CSV
df.to_csv('ENSO_dataset_preprocessed.csv', index=False)

print("Dataset berhasil disimpan sebagai SST_preprocessed.csv")

Dataset berhasil disimpan sebagai SST_preprocessed.csv


# Autoencoder

In [12]:
#pisahkan fitur SST dari target
# Ambil kolom grid SST (fitur)
X = df[grid_cols].values  # jadi numpy array
print("Shape data grid:", X.shape)  # (jumlah_sampel, 10991)

Shape data grid: (900, 10988)


In [13]:
#split data
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)
print("Train:", X_train.shape, "Validation:", X_val.shape)

Train: (720, 10988) Validation: (180, 10988)


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_dim = X_train.shape[1]   # 10991 fitur
encoding_dim = 128             # ukuran bottleneck (kompresi)

# Encoder
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(2048, activation='relu')(input_layer)
encoded = layers.Dense(512, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)  # bottleneck

# Decoder
decoded = layers.Dense(512, activation='relu')(encoded)
decoded = layers.Dense(2048, activation='relu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)  # rekonstruksi

# Autoencoder & Encoder model
autoencoder = models.Model(input_layer, decoded)
encoder = models.Model(input_layer, encoded)

# Compile autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10988)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │    22,505,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2048)           │     1,050,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10988)          │    22,514,412 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,251,308 (180.25 MB)

 Trainable params: 47,251,308 (180.25 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
history = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=64,
    shuffle=True,
    validation_data=(X_val, X_val)
)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 400ms/step - loss: 0.0362 - val_loss: 0.0240
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 353ms/step - loss: 0.0232 - val_loss: 0.0232
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 345ms/step - loss: 0.0220 - val_loss: 0.0207
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 345ms/step - loss: 0.0205 - val_loss: 0.0192
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 343ms/step - loss: 0.0176 - val_loss: 0.0166
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 347ms/step - loss: 0.0158 - val_loss: 0.0159
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 347ms/step - loss: 0.0153 - val_loss: 0.0155
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 345ms/step - loss: 0.0146 - val_loss: 0.0149
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 364ms/step - loss: 0.0141 - val_loss: 0.0149
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 347ms/step - loss: 0.0139 - val_loss: 0.0147
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 349ms/step - loss: 0.0138 - val_loss: 0.0140
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 348ms/step

In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Rekonstruksi data validation
X_val_pred = autoencoder.predict(X_val)

# Hitung MSE dan RMSE
mse = mean_squared_error(X_val.flatten(), X_val_pred.flatten())
rmse = np.sqrt(mse)

print(f"Reconstruction MSE: {mse:.6f}")
print(f"Reconstruction RMSE: {rmse:.6f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Reconstruction MSE: 0.007909
Reconstruction RMSE: 0.088935


In [17]:
X_encoded = encoder.predict(X)
print("Shape data kompresi:", X_encoded.shape)  # (jumlah_sampel, 128)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Shape data kompresi: (900, 128)


# ADASYN

In [23]:
from imblearn.over_sampling import ADASYN

# Ambil fitur (hasil autoencoder)
X = X_encoded

# Ambil target asli (nilai NINO3.4 numerik)
y_numeric = df['NINO3.4'].values

# Buat label kategori untuk balancing
def categorize_nino34(anom):
    if anom >= 0.5:
        return 1   # El Niño
    elif anom <= -0.5:
        return -1  # La Niña
    else:
        return 0   # Netral

y_class = np.array([categorize_nino34(val) for val in y_numeric])

# Gabungkan X dan y_numeric supaya keduanya ikut di-resample
data = np.hstack([X, y_numeric.reshape(-1, 1)])

# Resample berdasarkan y_class (kategori) tapi tetap bawa NINO3.4 asli
adasyn = ADASYN(random_state=42)
data_resampled, y_class_resampled = adasyn.fit_resample(data, y_class)

# Pisahkan kembali
X_resampled = data_resampled[:, :-1]  # fitur (encoded)
y_numeric_resampled = data_resampled[:, -1]  # target asli (regresi)

print("Distribusi kelas sebelum:", np.bincount(y_class + 1))
print("Distribusi kelas sesudah:", np.bincount(y_class_resampled + 1))
print("Jumlah total sampel setelah balancing:", len(y_numeric_resampled))


Distribusi kelas sebelum: [421 299 180]
Distribusi kelas sesudah: [421 357 443]
Jumlah total sampel setelah balancing: 1221


In [24]:
# Gabungkan jadi satu DataFrame
balanced_df = pd.DataFrame(X_resampled)
balanced_df['NINO3.4'] = y_numeric_resampled

# Simpan ke CSV
balanced_df.to_csv("ENSO_dataset_balanced.csv", index=False)

print("Dataset balanced berhasil disimpan")

Dataset balanced berhasil disimpan
